In [5]:
from IPython import get_ipython
from IPython.display import display
import aiohttp
import asyncio
from datetime import datetime
import json
import pytz
import terminaltables
import os

timezone = pytz.timezone('Europe/Berlin')
datetime_format = "%d.%m.%Y %H:%M"

class EFA:
    def __init__(self, url, proximity_search=False):
        self.dm_url = url + "/XML_DM_REQUEST"
        self.dm_post_data = {
            "language": "de",
            "mode": "direct",
            "outputFormat": "JSON",
            "type_dm": "stop",
            "useProxFootSearch": "0",
            "useRealtime": "1",
        }

        if proximity_search:
            self.dm_post_data["useProxFootSearch"] = "1"

    async def get_departures(self, place, name, ts):
        self.dm_post_data.update(
            {
                "itdDateDay": ts.day,
                "itdDateMonth": ts.month,
                "itdDateYear": ts.year,
                "itdTimeHour": ts.hour,
                "itdTimeMinute": ts.minute,
                "name_dm": name,
            }
        )
        if place is None:
            self.dm_post_data.pop("place_dm", None)
        else:
            self.dm_post_data.update({"place_dm": place})
        departures = list()
        async with aiohttp.ClientSession() as session:
            async with session.post(self.dm_url, data=self.dm_post_data) as response:
                departures = json.loads(await response.text())
        return departures


async def main():
    now = datetime.now()
    departures = await EFA("https://efa.vrr.de/standard/").get_departures(
        "Gelsenkirchen", "HBF", now                                                  # HIER VRR HALTESTELLE EINTRAGEN
    )
    #print(json.dumps(departures))
    #with open('data.json', 'w') as outfile:
     #   json.dump(departures, outfile)
    return departures

def display_departure(departures):
    line = departures["departureList"][0]["servingLine"]["number"]
    route = departures["departureList"][0]["servingLine"]["direction"]
    deptime = getDateTime(departures["departureList"][0]["dateTime"])

    if departures["departureList"][0]["servingLine"]["delay"] != "0":
        delay = departures["departureList"][0]["servingLine"]["delay"]
        print(line, route, deptime, delay)
    else:
        print(line, route, deptime)

def displayall(departures):
    stop = departures["dm"]["points"]["point"]["name"]
    print(f"departures for: { stop }")
    for departure in departures["departureList"]:
        line = departure["servingLine"]["number"]
        route = departure["servingLine"]["direction"]
        deptime = getDateTime(departure["dateTime"]) # realDateTime = Incl Verstpätung, dateTime = Fahrplan
        platform = departure["platformName"]
        if "delay" in departure["servingLine"]:
            delay = departure["servingLine"]["delay"]
        else:
            delay = 0
        # commingin = deptime - getCurrentDate() # This variable was calculated but not used.
        countdown = departure["countdown"]

        if len(countdown) < 2:
            countdown = "0" + countdown

        if int(countdown) < 1:
            print(line, route, platform, deptime.strftime(datetime_format),delay, "sofort")
        if int(countdown) > 0 and int(countdown) < 60:
            print(line, route, platform, deptime.strftime(datetime_format), delay, f"in: {countdown} min")
        if int(countdown) < 120 and int(countdown) > 60:
             # This condition was printing without the countdown. It seems like the logic for the last case was missing the countdown part.
             # Assuming you want to show countdown even if it's 60 or more. Adjust as needed.
             print(line, route, platform, deptime.strftime(datetime_format), delay)

def getCurrentDate():
    now = datetime.now(timezone)
    return datetime(year=int(now.year),month=int(now.month),day=int(now.day),hour=int(now.hour),minute=int(now.minute),tzinfo=timezone)

def getDateTime(data):
    year = data["year"]
    month = data["month"]
    # weekday =  data["weekday"] # This variable was retrieved but not used.
    day = data["day"]
    hour = data["hour"]
    minute = data["minute"]

    date = datetime(year=int(year),month=int(month),day=int(day),hour=int(hour),minute=int(minute),tzinfo=timezone)
    return date

def displayalltable(rawdata):
    header = [["line","destination","platform","depature","delay","countdown"]]
    data = list(header)
    for departure in rawdata["departureList"]:
        line = departure["servingLine"]["number"]
        route = departure["servingLine"]["direction"]

        #realDateTime = Incl Verstpätung, dateTime = Fahrplan
        if "realDateTime" in departure:
            deptime = getDateTime(departure["realDateTime"])
        else:
            deptime = getDateTime(departure["dateTime"])
        platform = departure["platformName"]
        if "delay" in departure["servingLine"]:
            delay = departure["servingLine"]["delay"]
        else:
            delay = 0

        countdown = departure["countdown"]
        if len(countdown) < 2:
            countdown = "0" + countdown

        if int(countdown) < 1:
            package = [line,route,platform,deptime.strftime(datetime_format),delay,"now"]
            data.append(package)
        elif int(countdown) > 0 and int(countdown) < 60: # Changed to elif for clarity and correctness
            package = [line, route, platform, deptime.strftime(datetime_format), delay, f"in: {countdown} min"]
            data.append(package)
        else: # Handling the case when countdown is 60 or more
             package = [line, route, platform, deptime.strftime(datetime_format), delay, f"in: {countdown} min"] # Assuming you want to show countdown even if it's 60 or more. Adjust as needed.
             data.append(package)


    table = terminaltables.AsciiTable(data, title=rawdata["dm"]["points"]["point"]["name"])
    print(table.table)

# Run the async main function directly by awaiting it
data = await main()

# Process the data
displayalltable(data)

# The os.system("pause") command is specific to Windows and will not work on Linux.
# Since the code is intended for a Jupyter notebook which is likely on Linux, this line can be removed or commented out.
# os.system("pause")

+Gelsenkirchen, Hbf--+-------------------------------+----------------+------------------+-------+------------+
| line               | destination                   | platform       | depature         | delay | countdown  |
+--------------------+-------------------------------+----------------+------------------+-------+------------+
| S2                 | Dortmund Hbf                  | 7              | 11.07.2025 16:06 | 4     | in: 03 min |
| RE42               | Essen Hauptbahnhof            | 4              | 11.07.2025 16:08 | 4     | in: 05 min |
| 302                | Gelsenkirchen Rheinelbestr.   | U-Bahn Gleis 2 | 11.07.2025 16:04 | 0     | in: 01 min |
| 302                | GE Buer Rathaus               | U-Bahn Gleis 1 | 11.07.2025 16:05 | 0     | in: 02 min |
| 381                | Gelsenkirchen Kirschblütenweg | 02             | 11.07.2025 16:06 | 0     | in: 03 min |
| FLX 1348 FlixTrain | Hamburg Hbf                   | 7              | 11.07.2025 16:06 | 0     | in: 0